# **Funciones**

Este notebook está basado en el curso de [_Julia Academy_](https://github.com/JuliaAcademy/Introduction-to-Julia).

Temas:
1. Como declarar una función en Julia
2. "Duck-typing" en Julia
3. Funciones mutantes vs no mutantes en Julia
4. Algunas funciones de mayor orden

Una función es un bloque de código que podemos llamar para ser ejecutado en nuestro programa y ejecutar una tarea específica. Un ejemplo que hemos utilizado ya por ejemplo es la función `print`, que imprime en pantalla los argumentos que le demos. Aprenderemos ahora a escribir nuestras propias funciones y algunas características útiles que tienen en `Julia`.

## **Como declarar una función en Julia**
Julia tiene varias formas de declarar funciones. La primera es utilizando las palabras clave `function` y `end`.

In [1]:
function saludo(nombre)
    println("Hola $nombre, ¡qué bueno verte!")
end

saludo (generic function with 1 method)

`saludo` toma un argumento `nombre`, y llama a la función `print` para escribir un saludo.

In [2]:
function f(x)
    x^2
end
#Nota: podríamos haber escrito return x^2, y también habría funcionado
#Las funciones de Julia retornan el resultado de la última operación que
#hicieron.

f (generic function with 1 method)

`f` toma un argumento `x`, y devuelve el resultado de la operación `x^2`.

Podemos llamar a estas funciones de la siguiente manera:

In [3]:
saludo("Mario")

Hola Mario, ¡qué bueno verte!


In [4]:
f(49)

2401

También podemos definir funciones en una sola línea si es que son suficientemente cortas.

In [5]:
salu2(nombre) = println("Hola $nombre, ¡qué bueno verte!")

salu2 (generic function with 1 method)

In [6]:
f2(x) = x^2

f2 (generic function with 1 method)

Tanto `salu2` como `f2` son idénticas a `saludo` y `f`. Sólo las hemos definido de una forma más compacta.

In [7]:
salu2("María")

Hola María, ¡qué bueno verte!


In [8]:
f2(42)

1764

Finalmente, podríamos haberlas declarado como funciones "anónimas"

In [9]:
saludo3 = nombre -> println("Hola $nombre, ¡qué bueno verte!")

#1 (generic function with 1 method)

In [10]:
f3 = x -> x^2

#3 (generic function with 1 method)

In [11]:
saludo3("René")

Hola René, ¡qué bueno verte!


In [12]:
f3(42)

1764

El uso principal de las funciones anónimas es para dárselas a otras funciones que tomen funciones como argumento, y veremos ejemplos más adelante. Por el momento no se preocupe mucho por ellas.

Para definir funciones que acepten más argumentos, simplemente debemos escribir los argumentos en el orden que serán aceptados.

In [13]:
function maximo(x,y)
    return x > y ? x : y
    #===============================================================================
    Nota: aquí hemos utilizado un operador ternario. Esto es equivalente a haber
    escrito
    
    if x > y
        return x
    else
        return y
    end
    ================================================================================#
end
println(maximo("abc","ajk"))
println(maximo(2, 8))

ajk
8


## **Duck-typing in Julia**
*"Si hace cuak como un pato, es un pato."* <br><br>
Las funciones de Julia van a funcionar siempre que las operaciones estén bien definidas para sus argumentos. <br><br>
Por ejemplo, la función `saludo` que definimos anteriormente funciona si el argumento es un número

In [14]:
saludo(3.14)

Hola 3.14, ¡qué bueno verte!


Y `f` funcionará en una matriz. 

In [15]:
A = rand(3, 3) #Esta línea crea una matriz de 3x3 con elementos aleatorios entre 0 y 1
A

3×3 Matrix{Float64}:
 0.63243   0.8037    0.796123
 0.909436  0.252811  0.648774
 0.732501  0.500518  0.684106

In [16]:
f(A)  #f(A) = A^2 = A*A. Notar que la operación que realizó Julia fue la multiplicación
      #de matrices, no la multiplicación elemento por elemento.

3×3 Matrix{Float64}:
 1.71404  1.10994  1.56954
 1.2803   1.11955  1.33187
 1.41955  1.05765  1.37588

`f` también va a funcionar con un `string`, ya que `^` es el operador utilizado para repetir strings.

In [17]:
f("hola")  #"hola"^2 = "hola"*"hola" = "holahola"

"holahola"

Sin embargo, `f` no va a funcionar en un vector. Esto es porque no podemos multiplicar vectores columna por vectores columna.

In [18]:
v = rand(3)

3-element Vector{Float64}:
 0.7158892670845735
 0.06611679545610871
 0.07426176035274323

In [19]:
# Esta línea dará error, ya que v es un vector columna, y v*v no está bien definido.
f(v)

LoadError: MethodError: no method matching ^(::Vector{Float64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at ~/soft/julia-1.7.2/share/julia/base/strings/basic.jl:721
[0m  ^([91m::Rational[39m, ::Integer) at ~/soft/julia-1.7.2/share/julia/base/rational.jl:475
[0m  ^([91m::LinearAlgebra.Diagonal[39m, ::Integer) at ~/soft/julia-1.7.2/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:196
[0m  ...

## **Funciones mutantes vs. no-mutantes**

A esta altura puede que haya notado que algunas funciones en los tutoriales anteriores tienen un signo `!`. Por convención, las funciones que terminan en un signo `!` pueden alterar el contenido de sus argumentos, y las que no lo tienen no.

Por ejemplo, miremos la diferencia entre las funciones de ordenamiento `sort` y `sort!`

In [23]:
v = [3, 5, 2] #Creamos un vector v

3-element Vector{Int64}:
 3
 5
 2

In [24]:
sort(v)      #Vemos qué retorna la función sort(v)

3-element Vector{Int64}:
 2
 3
 5

In [25]:
v            #Comprobamos que v no fue modificado

3-element Vector{Int64}:
 3
 5
 2

`sort(v)` retorna un arreglo ordenado de los elementos de `v`, pero `v` no cambia. <br><br>

Por otro lado, si usamos `sort!(v)`, los contenidos de `v` son ordenados dentro del mismo arreglo.

In [26]:
sort!(v)    #Vemos qué retorna la función sort!(v)

3-element Vector{Int64}:
 2
 3
 5

In [27]:
v

3-element Vector{Int64}:
 2
 3
 5

**Importante**
Es una buena práctica usar esta convención al definir funciones, de manera de saber rápido si la función que está utilizando corre el riesgo de modificar vectores que ha definido. 

## **Comportamiento de las funciones ante arreglos y diccionarios** ###

Veamos qué sucede si intentamos modificar una variable dentro de una función:

In [29]:
function duplicar(x)
    x = 2*x
end
a = 2
b = duplicar(a)
println("a = $a, b = $b")

a = 2, b = 4


Notamos que `a` no fue modificada por la función `duplicar`, pese a que internamente realizamos la operación `x = 2*x`. Esto es porque `duplicar` utilizó internamente una copia de `a`, y por lo tanto no modificó la variable original.

Por el contrario, cuando una función recibe como argumento una estructura de datos mutable, puede modificarla. Por ejemplo, la siguiente función duplica los elementos de un arreglo:

In [31]:
function duplicar!(arreglo)
    for i in 1:size(arreglo,1) #El "1" indica que nos interesa el tamaño en la primer dimensión del arreglo
        arreglo[i] *= 2
        #=======================================================================
        arreglo[i] *= 2 es una forma compacta de escribir
        arreglo[i] = arreglo[i]*2
        Esta notación también funciona para otras operaciones como +, - y /
        =======================================================================#
    end
    return arreglo #Esta línea no es necesaria, solo la agregamos para que
                   #la función retorne algo
end

duplicar! (generic function with 1 method)

In [32]:
duplicar!([1,2,3])

3-element Vector{Int64}:
 2
 4
 6

In [40]:
w = copy(v)

3-element Vector{Int64}:
 56
  4
  6

In [41]:
w[1] = 89
v

3-element Vector{Int64}:
 56
  4
  6

In [37]:
v = [1,2,3]
duplicar!(v)
print(v)

[2, 4, 6]

Como somos buenas personas y no queremos confundir a nadie, agregamos un signo `!`, avisando que nuestra función modificará el argumento de entrada que le demos.
Si no tiene nada mejor que hacer en este momento, pruebe crear una función `duplicar` que retorne el un arreglo multiplicado por 2 sin modificarlo.

También podemos modificar elementos de un diccionario. Por ejemplo, la siguiente función agrega una entrada a un diccionario:

In [35]:
function agregar_numero!(diccionario, contacto, numero)
    diccionario[contacto] = numero #agregamos un contacto a un diccionario
end
agenda = Dict("Juan" => 5551234, "Martina" => 5558954)
agregar_numero!(agenda, "Sofía", 5559812)
agenda

Dict{String, Int64} with 3 entries:
  "Sofía"   => 5559812
  "Juan"    => 5551234
  "Martina" => 5558954

Esto es muy útil, ya que Julia no necesita crear copias de los arreglos para trabajar dentro de las funciones, pero es también importante tenerlo en cuenta para no cometer errores y modificar un arreglo cuando no queremos hacerlo.

## **Algunas funciones de mayor orden**

### `map`

`map` es una función de "mayor orden" de Julia que *toma una función* como uno de sus argumentos. `map` luego aplica esa función a todos los elementos de una estructura de datos que le pasemos.
Por ejemplo, ejecutar

```julia
map(f, [1, 2, 3])
```
retornará un arreglo donde `f` fue aplicada a todos los elementos de `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [42]:
map(f, [1, 2, 3])   #Recordemos que más arriba definimos f(x) = x^2

3-element Vector{Int64}:
 1
 4
 9

Es decir, elevamos al cuadrado cada elemento de `[1, 2, 3]`.

También podríamos haberle dado a `map` una función anónima,

In [43]:
map(x -> x^3, [1, 2, 3]) #x -> x^3 es una función anónima, es decir, no tiene nombre.
                         #La definimos únicamente para pasarla como argumento a la
                         #función map

3-element Vector{Int64}:
  1
  8
 27

y ahora hemos elevado al cubo los elementos de `[1, 2, 3]`

### `broadcast`

`broadcast` es otra función de mayor orden como `map`. `broadcast` es una generaliación de `map`, por lo que puede hacer lo mismo que `map` y mucho más. La sintaxis básica de `broadcast` es la misma que para `map`

In [44]:
broadcast(f, [1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Y nuevamente aplicamos `f` a todos los elementos de `[1, 2, 3]`

`broadcast` se diferencia de `map` en cómo se comporta ante multiples argumentos de distintas dimensiones. `broadcast` intentará encontrar una dimensión en común y `map` no. Por ejemplo, notar la diferencia entre estas dos líneas:

In [45]:
map(+, 1, [1,2,3]) #Nota: Los operadores de Julia son funciones. 
                   #      Escribir +(a,b) es lo mismo que escribir a+b.

1-element Vector{Int64}:
 2

In [46]:
broadcast(+,1,[1,2,3])

3-element Vector{Int64}:
 2
 3
 4

Una abreviatura muy útil de `broadcast` es colocar `.` entre una función a la que quiera hacer `broadcast` y sus argumentos. Por ejemplo,

```julia
broadcast(f, [1, 2, 3])
```
es equivalente a
```julia
f.([1, 2, 3])
```

In [47]:
f.([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

Note nuevamente como esto es distinto a llamar
```julia
f([1, 2, 3])
```
Podemos elevar al cuadrado cada elemento del vector, ¡pero esto no es lo mismo que elevar al cuadrado el vector, ya que esto no está definido!

Para mostrar más claramente la diferencia, considere

```julia
f(A)
```
y
```julia
f.(A)
```
para una matriz `A`:

In [48]:
A = [i + 3*j for j in 0:2, i in 1:3]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [49]:
f(A)

3×3 Matrix{Int64}:
  30   36   42
  66   81   96
 102  126  150

En este caso estamos haciendo
```
f(A) = A^2 = A * A
``` 

Por otro lado,

In [51]:
B = f.(A)

3×3 Matrix{Int64}:
  1   4   9
 16  25  36
 49  64  81

contiene los cuadrados de cada una de las entradas individuales de `A`.

Esta sintaxis nos permite escribir expresiones complejas de una forma mucho más natural

In [52]:
A .+ 2 .* f.(A) ./ A

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

en lugar de

In [53]:
broadcast(x -> x + 2 * f(x) / x, A)

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

y ambas expresiones van a tener la misma performance.

In [55]:
@. A + 2 * f(A) / A

3×3 Matrix{Float64}:
  3.0   6.0   9.0
 12.0  15.0  18.0
 21.0  24.0  27.0

## **Especificando tipos**

Al definir una función, podemos especificar el `tipo` de argumentos que recibe. Incluso, como veremos más adelante en multiple dispatch, podemos hacer que una función se comporte distinto dependiendo del `tipo` de los argumentos que recibe. La sintaxis para especificar que una variable `a` tiene tipo `Type` es `a::Type`.
En el siguiente ejemplo definimos una función que únicamente funciona si el argumento es un número entero

In [56]:
function factorial(n::Int)
    if n < 1
        println("Error, n < 1. Intente con un número positivo.")
        return -1
    end
    result = 1
    for i in 2:n
        result *= i
    end
    return result
end

factorial (generic function with 1 method)

In [57]:
factorial(3)

6

In [58]:
factorial(3.0)

LoadError: MethodError: no method matching factorial(::Float64)
[0mClosest candidates are:
[0m  factorial([91m::Int64[39m) at In[56]:1

## **Sobre los argumentos de las funciones**

### **Cantidad arbitraria de variables de argumentos**

La función `print` tiene un comportamiento interesante que quizás haya notado. Podemos darle una cantidad arbitraria de argumentos, y `print` imprime cada uno de ellos:

In [59]:
print("hola", " yo", " estoy", " dándole", " muchos", " argumentos", " a", " print")

hola yo estoy dándole muchos argumentos a print

In [61]:
((1,2,3)...,4,5,6)

(1, 2, 3, 4, 5, 6)

Esto se logra utilizando el operador _splat_. Aprenderemos de el creando nuestra propia función con una cantidad _variable_ de argumentos. La función `suma_arbitraria` suma una cantidad arbitraria de argumentos:

In [62]:
function suma_arbitraria(x...)
    println("Argumentos dados: $x")
    return sum(x)
end

suma_arbitraria (generic function with 1 method)

In [64]:
suma_arbitraria(1,2,3,4,8,18352)

Argumentos dados: (1, 2, 3, 4, 8, 18352)


18370

Los elementos de "x" fueron leidos como una tupla. Podríamos tambien haber dado como argumento una tupla o un arreglo:

In [66]:
suma_arbitraria([i for i in 1:10]...)

Argumentos dados: (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


55

Note que si buscamos la documentación de `print`, el segundo argumento de la función es `xs...`.

In [67]:
?print()

```
print([io::IO], xs...)
```

Write to `io` (or to the default output stream [`stdout`](@ref) if `io` is not given) a canonical (un-decorated) text representation. The representation used by `print` includes minimal formatting and tries to avoid Julia-specific details.

`print` falls back to calling `show`, so most types should just define `show`. Define `print` if your type has a separate "plain" representation. For example, `show` displays strings with quotes, and `print` displays strings without quotes.

See also [`println`](@ref), [`string`](@ref), [`printstyled`](@ref).

# Examples

```jldoctest
julia> print("Hello World!")
Hello World!
julia> io = IOBuffer();

julia> print(io, "Hello", ' ', :World!)

julia> String(take!(io))
"Hello World!"
```


### **Argumentos opcionales**
Podemos dar valores por defecto a ciertas variables, lo cual puede ayudar a quien use la función a evitar dar argumentos innecesarios. La siguiente función retorna la fecha en el formato `(día, mes, año)`, y asume que es enero del año 2000 si no especificamos mes ni año: 

In [71]:
function fecha(dd, mm = 1, aa = 2000)
    return (dd, mm, aa)
end

fecha (generic function with 3 methods)

In [72]:
fecha()    #Esto no funcionará, porque debemos especificar el día

LoadError: MethodError: no method matching fecha()
[0mClosest candidates are:
[0m  fecha([91m::Any[39m) at In[71]:1
[0m  fecha([91m::Any[39m, [91m::Any[39m) at In[71]:1
[0m  fecha([91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at In[71]:1

In [74]:
fecha(9) #Aquí, dd = 10, y mm = 1, aa = 2000 por defecto

(9, 1, 2000)

In [76]:
fecha(9,3) #Aquí hemos especificado el mes, pero no el año

(9, 3, 2000)

In [77]:
fecha(9,3,2022) #Finalmente, aquí especificamos tanto el día como el mes y el año

(9, 3, 2022)

In [79]:
fecha((9,3)...)  #Como dato extra, también podemos usar el operador ... para ingresar
                  #una tupla como argumentos separados en cualquier función

(9, 3, 2000)

### **Argumentos Keyword** (Palabras clave) 

Un problema con la función que definimos anteriormente, es que debemos escribir los argumentos en el orden específico que los espera la función. No solo eso, sino que podemos ahorrarnos escribir el mes y no escribir el año, pero no podemos escribir el año e ignorar el mes. Para solucionar este problema, podemos usar `keyword arguments` tras un signo `;`.

In [107]:
function fecha_v2(dd; mm = 1, aa = 2000, format = :dma)
    if format == :dma
        return (dd, mm, aa)
    elseif format == :mda
        return (mm, dd, aa)
    elseif format == :amd
        return (aa, mm, dd)
    else
        println("Formato de fecha no reconocido. Solo acepto dma, mda y amd")
    end
end

fecha_v2 (generic function with 1 method)

In [108]:
mda = 2

2

In [106]:
fecha_v2(1;format = :mda, mm = 3) #El ; solo lo usamos para definir la función, no para llamarla

(3, 1, 2000)

Ahora podemos escribir solo los argumentos que nos interesen y en el orden que querramos, aunque debemos especificar el nombre del argumento.

In [96]:
function funcion_kwargs(argumento_1, y...;x...)
    println("el primer argumento vale = $argumento_1")
    println("Usted ingresó los siguientes argumentos keyword")
    
    for element in y
        println(element)
    end
    for (key, val) in x
        println("$key = $val")
    end
end

perro = "perro"

funcion_kwargs(1,2,3,4, a = 3, b = "manzana", perro)

#Note que cualquier cosa luego de un ; será considerado parte de los kwargs!
#La línea anterior sin ; no funciona

el primer argumento vale = 1
Usted ingresó los siguientes argumentos keyword
2
3
4
perro
a = 3
b = manzana


## **Cómo agregar documentación**
Para documentar una función, simplemente debemos agregar un string antes de la declaración de la función.

In [97]:
"""
    funciondocumentada(x...)

# Ejemplo de documentación

Esta función existe únicamente para explicar cómo agregar documentación.
Escribir documentación funciona como escribir en modo **markdown**.

Note que para separar líneas debe poner una línea vacía entre medio.

Puede llamarla utilizando `funciondocumentada(argumentos)`.\n

Acepta una cantidad arbitraria de argumentos.
"""
function funciondocumentada(x...)
    pass
end

funciondocumentada

In [98]:
@doc funciondocumentada

```
funciondocumentada(x...)
```

# Ejemplo de documentación

Esta función existe únicamente para explicar cómo agregar documentación. Escribir documentación funciona como escribir en modo **markdown**.

Note que para separar líneas debe poner una línea vacía entre medio.

Puede llamarla utilizando `funciondocumentada(argumentos)`.

Acepta una cantidad arbitraria de argumentos.


In [99]:
using JLD2

In [100]:
@doc jldsave

```
jldsave(filename, compress=false; kwargs...)
```

Creates a JLD2 file at `filename` and stores the variables given as keyword arguments.

# Examples

```
jldsave("example.jld2"; a=1, b=2, c)
```

is equivalent to

```
jldopen("example.jld2, "w") do f
    f["a"] = 1
    f["b"] = 2
    f["c"] = c
end
```

To choose the io type `IOStream` instead of the default `MmapIO` use  `jldsave{IOStream}(fn; kwargs...)`.


## **Ejercicios opcionales**
***1)*** Escriba una función `suma_uno` que sume 1 a su argumento

In [ ]:
@assert suma_uno(1) == 2

In [ ]:
@assert suma_uno(11) == 12

***2)*** Use `map` o `broadcast` para incrementar cada elemento de una matriz `A` en `1` y asígnelo a una variable `A1`.

In [ ]:
@assert A1 == [2 3 4; 5 6 7; 8 9 10]

***3)***
Use la sintaxis `.` de `broadcast` para incrementar todos los valores de `A1` en `1` y guarde el resultado en `A2`

In [ ]:
@assert A2 == [3 4 5; 6 7 8; 9 10 11]

***4)*** Cree una función `misuma(vec_1, vec_2)` que retorne la suma directa de `vec_1, vec_2`. Use para esto un loop `for` dentro de la función.
Luego cree otra función llamada `misuma!(vec_r, vec_1, vec_2)` que funcione igual, pero guarde el resultado en `vec_r`.

In [ ]:
#= 
function misuma(vec_1, vec_2)
    *código*
end
=#

In [ ]:
#= 
function misuma!(vec_r, vec_1, vec_2)
    *código*
end
=#

In [ ]:
#Verificación de misuma. Si esta celda no da error, misuma funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
#Verificación de misuma!. Si esta celda no da error, misuma! funciona.
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end

***5)*** Repita el ejercicio anterior, pero usando broadcast para escribir una función más compacta

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]

for elemento in misuma(vec_1, vec_2)
    @assert elemento == 4
end

In [ ]:
vec_1 = [1,2,3]
vec_2 = [3,2,1]
vec_r = copy(vec_1)
misuma!(vec_r, vec_1, vec_2)
for elemento in vec_r
    @assert elemento == 4
end